In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import pickle

from torchmetrics.regression import SymmetricMeanAbsolutePercentageError
import torch
import os
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
import numpy as np

In [1]:
def metric(y_pred, y_test):
    ad_smape = SymmetricMeanAbsolutePercentageError()
    smape_adjust = torch.mean(torch.stack([ad_smape(y_pred.detach().cpu()[i], y_test.detach().cpu()[i]) * 0.5 for i in range(len(y_test))]))
    return smape_adjust

In [3]:
data_path = '/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/'
meta_df = pd.read_csv(os.path.join(data_path,'meta_data_image_text_nofilter.csv'), index_col='item_number')
meta_df = meta_df.drop(['sales_std'], axis=1)
# meta_df = pd.read_csv(os.path.join(data_path,'240109_all_meta_sales_total.csv'), index_col='item_number')
# meta_df = meta_df.drop(['sales_total','sales_std'], axis=1)
meta_df

,sales_mean,fabric_WOVEN,fabric_DENIM,fabric_JERSEY,fabric_SWEATER,fabric_특종,main_color_MULTI_COLOR,main_color_BLUE,main_color_WHITE,main_color_PURPLE,...,main_color_RED,main_color_BROWN,main_color_GREY,main_color_YELLOW,main_color_ORANGE,category_OP,category_DP,category_HP,category_PT,category_WP
item_number,,,,,,,,,,,,,,,,,,,,,
JPOP123A,4.833333,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
JPOP123B,5.833333,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
JPOP123C,6.166667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
JPOP123D,1.083333,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
JPOP221A,0.500000,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MWPT71A1,65.666667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
MWPT71A2,49.666667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
MWPT71A3,13.166667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
# for n_iter in range(1,6):
n_iter = 1
orig_df = pd.read_csv("/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/qcut_value_df_bin10.csv", index_col='품번')
n_classes = orig_df['qcut_label'].nunique()
test_list = pickle.load(open("/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/12salesweek_test_item_number296.pkl", 'rb')).drop("MTPT6102")[:]
pred_list = []
repr_list = []

# qcut_df.loc[qcut_df['qcut_label'] != 0, 'qcut_label'] = 1

# n_iter = 4
for idx in range(1, n_iter+1):
    qcut_df = orig_df.copy()

    zero_idx = qcut_df['qcut_label'].isin(range(idx))
    qcut_df.loc[zero_idx, 'qcut_label'] = 0
    qcut_df.loc[~zero_idx, 'qcut_label'] = 1
    orig_df = orig_df[~zero_idx]
    qcut_label_mean = qcut_df.groupby('qcut_label')['sales_mean'].mean().values
    qcut_label_median = qcut_df.groupby('qcut_label')['sales_mean'].median().values

    df = meta_df.drop('sales_mean', axis=1)
    df = meta_df.merge(qcut_df['qcut_label'], left_index=True, right_index=True)

    train_df = df[~df.index.isin(test_list)]
    test_df = df[df.index.isin(test_list)]

    X_train = train_df.drop(['qcut_label', 'sales_mean'], axis=1).values
    y_train = train_df['qcut_label'].values

    if idx == 1:
        X_test = test_df.drop(['qcut_label', 'sales_mean'], axis=1).values
        y_test = test_df['sales_mean'].values
    
    X_test_binary = test_df.drop(['qcut_label', 'sales_mean'], axis=1).values
    y_test_binary = test_df['qcut_label'].values

    # model = LogisticRegression()
    # model.fit(X_train, y_train)
    model = KNeighborsClassifier(n_neighbors=7)
    n_over_samples = train_df['qcut_label'].value_counts().max()
    sm = SMOTE(sampling_strategy={i:n_over_samples for i in range(2)}, k_neighbors=5)
    X_train_os, y_train_os = sm.fit_resample(X_train, y_train)
    model.fit(X_train_os, y_train_os)
    
    y_pred_binary = model.predict(X_test_binary)
    y_pred = model.predict(X_test)

    print(np.unique(y_train, return_counts=True))
    print(np.unique(y_train_os, return_counts=True))
    print('ground-truth')
    print(y_test_binary)

    print('prediction')
    print(y_pred_binary)
    print(f'{idx}-split acc\t{accuracy_score(y_test_binary, y_pred_binary)}')
    print(f'{idx}-split rec\t{recall_score(y_test_binary, y_pred_binary)}')
    print(f'{idx}-split pre\t{precision_score(y_test_binary, y_pred_binary)}')
    print(f'{idx}-split f1\t{f1_score(y_test_binary, y_pred_binary)}')
    print()

    pred_list.append(y_pred)
    repr_list.append((qcut_label_mean[0], qcut_label_median[0]))
    if idx == n_iter:
        repr_list.append((qcut_label_mean[1], qcut_label_median[1]))

pred_df = pd.DataFrame(pred_list).T
pred_df['pred'] = n_iter
pred_df['mean_pred'] = repr_list[-1][0]
pred_df['median_pred'] = repr_list[-1][1]

for idx in range(n_iter-1, -1, -1):
    pred_df.loc[pred_df.iloc[:,idx] == 0, 'pred'] = idx
    pred_df.loc[pred_df.iloc[:,idx] == 0, 'mean_pred'] = repr_list[idx][0]
    pred_df.loc[pred_df.iloc[:,idx] == 0, 'median_pred'] = repr_list[idx][1]

gt = torch.FloatTensor(y_test)
mean_pred = torch.FloatTensor(pred_df['mean_pred'])
median_pred = torch.FloatTensor(pred_df['median_pred'])

print(f'{n_iter}-split: {metric(mean_pred, gt)}, {metric(median_pred, gt)}')

(array([0, 1]), array([1267,  174]))
(array([0, 1]), array([1267, 1267]))
ground-truth
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1
 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 1 0 0 0]
prediction
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 

In [10]:
pred_df

,0,1,2,pred,mean_pred,median_pred
0,0,0,0,0,36.659882,19.083333
1,0,0,0,0,36.659882,19.083333
2,0,0,1,0,36.659882,19.083333
3,0,0,1,0,36.659882,19.083333
4,0,0,1,0,36.659882,19.083333
...,...,...,...,...,...,...
290,0,0,1,0,36.659882,19.083333
291,1,0,1,1,243.025000,239.000000
292,0,0,1,0,36.659882,19.083333
293,0,0,0,0,36.659882,19.083333
